<a href="https://colab.research.google.com/github/Tbharvesh/DeepLearningProject/blob/main/DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# !unzip -q dataset.zip
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
%cd https://drive.google.com/drive/folders/11jXIcJ6MF3R_FbeMQigZ5PeQSEpYmurO?usp=drive_link


[Errno 2] No such file or directory: 'https://drive.google.com/drive/folders/11jXIcJ6MF3R_FbeMQigZ5PeQSEpYmurO?usp=drive_link'
/content


In [8]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Example for loading image datasets
dataset = image_dataset_from_directory(
    '/content/drive/My Drive/Covid19-dataset',
    image_size=(256, 256),
    batch_size=32)


Found 317 files belonging to 2 classes.
